In [1]:
import sys,os
sys.path.append("..")
import django
django.setup()
import datetime
import pandas as pd
import requests
from io import StringIO
# pd.set_option("display.max_rows", None)

In [ ]:
"""
單季
STEP: DATA
STOCK_ID: 3293
RPT_CAT: XX_M_QUAR
QRY_TIME: 20194

"""

"""
累季
STEP: DATA
STOCK_ID: 3293
RPT_CAT: XX_M_QUAR_ACC
QRY_TIME: 20194

"""

In [53]:
class CrawlGoodinfoFRTW:
    
    def __init__(self,date):
        self.date=date
    
    @staticmethod
    def season_to_date(sym):
        year=int(sym[:4])
        season=sym[-2:]
        if season == 'Q4':
            date=datetime.date(year + 1, 3, 31)
        elif season == 'Q3':
            date=datetime.date(year, 8, 14)
        elif season == 'Q2':
            date=datetime.date(year, 11, 14)    
        elif season == 'Q1':
            date=datetime.date(year, 5, 15)
        return date
    
    @staticmethod
    def get_query_time(date):
        year=date.year
        season=None
        if date.month >= 3 and date.month < 5:
            season = '4'
            year = year - 1
        elif date.month == 5:
            season = '1'
        elif date.month == 8:
            season = '2'
        elif date.month == 11:
            season = '3'
        else:
            return None
        if season:
            QRY_TIME=str(year)+season
        return QRY_TIME    
    
    def crawl_report(self,stock_id,RPT_CAT='XX_M_QUAR'):
        headers = {
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
            (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36',
            'content-type': 'application/x-www-form-urlencoded;',
            'origin': 'https://goodinfo.tw',
            'referer': 'https://goodinfo.tw/StockInfo/StockFinDetail.asp?'
        }
        QRY_TIME=self.get_query_time(self.date)
        if QRY_TIME is None:
            return None
        url='https://goodinfo.tw/StockInfo/StockFinDetail.asp?STEP=DATA&STOCK_ID='+stock_id+'&RPT_CAT='+RPT_CAT+'&QRY_TIME='+QRY_TIME

        r = requests.get(url, headers=headers)
        r.encoding = 'utf-8'
        html_df = pd.read_html(StringIO(r.text))
        df=pd.DataFrame(html_df[1])
        df.columns=df.iloc[0]
        df=df.iloc[1:].set_index('獲利能力')
        df=df.T
        drop_columns=[s for s in df.columns if '年預估' in s or '母公司' in s]
        df=df.drop(columns=drop_columns)
        
        df.columns=[s[:s.index("\xa0")]  if "\xa0" in s else s for s in df.columns]
        df=df.apply(lambda s: pd.to_numeric(s, errors='coerce'))
        df=df.reset_index().dropna(how='all',axis=1)
        df=df.rename(columns={0:'date'})
        df['stock_id']=stock_id
        df['date']=df['date'].apply(lambda s:self.season_to_date(s))
        return df

date=datetime.date(2020,4,3)    
df=CrawlGoodinfoFRTW(date).crawl_report('2330')
df

,date,營業毛利率,營業利益率,稅前淨利率,稅後淨利率,每股稅前盈餘,每股稅後盈餘,每股淨值,股東權益報酬率,資產報酬率,...,每股融資現金流量,每股淨現金流量,每股自由現金流量,負債對淨值比率,長期資金適合率,所得稅佔稅前淨利比率,業外損益佔營收比率,業外損益佔稅前淨利比率,財報評分,stock_id
0,2020-03-31,50.18,39.16,40.59,36.59,4.97,4.48,62.53,7.23,5.28,...,-0.66,0.11,1.21,39.62,120.3,9.86,1.43,3.52,96,2330
1,2019-08-14,47.57,36.82,38.33,34.50,4.33,3.90,61.20,6.44,4.62,...,-8.84,-7.61,1.29,34.44,132.7,10.00,1.52,3.96,96,2330
2,2019-11-14,43.04,31.66,33.42,27.71,3.10,2.57,59.92,4.05,3.02,...,-0.04,0.16,0.12,44.06,136.9,17.10,1.76,5.27,89,2330
3,2019-05-15,41.31,29.38,31.18,28.07,2.63,2.37,67.21,3.59,2.87,...,-0.87,2.62,3.42,25.46,157.9,9.97,1.79,5.74,89,2330
4,2019-03-31,47.67,36.97,38.33,34.51,4.29,3.86,64.67,6.15,4.93,...,0.55,3.44,2.78,24.60,161.8,9.97,1.37,3.56,98,2330
5,2018-08-14,47.39,36.58,37.99,34.22,3.82,3.44,60.75,5.81,4.43,...,-6.66,-5.54,1.18,25.01,155.7,9.91,1.40,3.69,96,2330
6,2018-11-14,47.81,36.19,37.55,30.99,3.38,2.79,57.46,4.67,3.52,...,-1.13,2.10,2.69,37.75,152.2,17.46,1.35,3.61,96,2330
7,2018-05-15,50.33,39.03,40.29,36.19,3.85,3.46,61.93,5.74,4.44,...,-2.22,0.94,3.36,27.55,160.1,10.16,1.26,3.12,96,2330
8,2018-03-31,49.99,39.23,40.23,35.78,4.31,3.83,58.70,6.72,5.18,...,0.37,5.61,5.44,30.81,152.0,11.08,1.00,2.49,96,2330
9,2017-08-14,49.93,38.89,39.87,35.68,3.88,3.47,55.18,6.49,4.70,...,-8.21,-6.27,1.97,28.64,142.9,10.51,0.98,2.45,96,2330


In [55]:
len(set(df.columns))

100

In [ ]:
en=pd.read_csv('/Users/benbilly3/Downloads/無標題的試算表 - 工作表1.csv')
en['col']

In [ ]:
pd.set_option("display.max_columns", None)
# dd=df.rename(columns={i:1 for i in c})
# dd
df

In [ ]:
[i for i in df2.columns  if i not in df.columns]

In [ ]:
v= filter(str.isalpha, '每股融資現金流量 (元)')
address = ''.join(list(v))
address

In [ ]:
'其他負債\xa0(%)'.replace('\xa0(%)','')

In [ ]:

def season_to_date(sym):
    year=int(sym[:4])
    season=sym[-2:]
    if season == 'Q4':
        date=datetime.date(year + 1, 3, 31)
    elif season == 'Q3':
        date=datetime.date(year, 8, 14)
    elif season == 'Q2':
        date=datetime.date(year, 11, 14)    
    else :
        date=datetime.date(year, 5, 15)
    return date
season_to_date('2019Q4')

In [ ]:
df=df.apply(lambda s: pd.to_numeric(s, errors='coerce'))
df=df.reset_index().dropna(how='all',axis=1)
df

In [22]:
list(df.columns)

['date',
 '營業毛利率',
 '營業利益率',
 '稅前淨利率',
 '稅後淨利率',
 '稅後淨利率\xa0(母公司)',
 '每股稅前盈餘\xa0(元)',
 '每股稅後盈餘\xa0(元)',
 '每股淨值\xa0(元)',
 '股東權益報酬率\xa0(當季)',
 '股東權益報酬率\xa0(年預估)',
 '資產報酬率\xa0(當季)',
 '資產報酬率\xa0(年預估)',
 '營收季成長率',
 '毛利季成長率',
 '營業利益季成長率',
 '稅前淨利季成長率',
 '稅後淨利季成長率',
 '稅後淨利季成長率\xa0(母公司)',
 '每股稅後盈餘季成長率',
 '營收年成長率',
 '毛利年成長率',
 '營業利益年成長率',
 '稅前淨利年成長率',
 '稅後淨利年成長率',
 '稅後淨利年成長率\xa0(母公司)',
 '每股稅後盈餘年成長率',
 '現金\xa0(%)',
 '應收帳款\xa0(%)',
 '存貨\xa0(%)',
 '速動資產\xa0(%)',
 '流動資產\xa0(%)',
 '基金與投資\xa0(%)',
 '固定資產\xa0(%)',
 '無形資產\xa0(%)',
 '其他資產\xa0(%)',
 '現金季成長率',
 '應收帳款季成長率',
 '存貨季成長率',
 '流動資產季成長率',
 '基金與投資季成長率',
 '固定資產季成長率',
 '無形資產季成長率',
 '其他資產季成長率',
 '資產總額季成長率',
 '現金年成長率',
 '應收帳款年成長率',
 '存貨年成長率',
 '流動資產年成長率',
 '基金與投資年成長率',
 '固定資產年成長率',
 '無形資產年成長率',
 '其他資產年成長率',
 '資產總額年成長率',
 '應付帳款\xa0(%)',
 '流動負債\xa0(%)',
 '長期負債\xa0(%)',
 '其他負債\xa0(%)',
 '負債總額\xa0(%)',
 '普通股股本\xa0(%)',
 '股東權益總額\xa0(%)',
 '應付帳款季成長率',
 '流動負債季成長率',
 '長期負債季成長率',
 '其他負債季成長率',
 '負債總額季成長率',
 '普通股股本季成長率',
 '股東權益總額季成長率',
 '應付帳款年成長率',
 '流動負債年成長率',
 '

In [ ]:
class CrawlFinancialReportTW:
    def __init__(self,date):
        self.date=date
        self.target_name = "台股月營收資訊"
        self.sub_market = ["sii", "otc", "rotc"]


    def crawl_main(self):
        url_date = last_month(self.date)
        data = []
        for i in self.sub_market:

            url = 'https://mops.twse.com.tw/nas/t21/' + i + '/t21sc03_' + str(url_date.year - 1911) + '_' + str(
                url_date.month) + '.html'

            # 偽瀏覽器
            headers = {
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko)'
                              ' Chrome/39.0.2171.95 Safari/537.36'}

            # 下載該年月的網站，並用pandas轉換成 dataframe
            try:
                r = requests.get(url, headers=headers)
                r.encoding = 'big5'
                html_df = pd.read_html(StringIO(r.text))
                # 處理一下資料
                if html_df[0].shape[0] > 500:
                    df = html_df[0].copy()
                else:
                    df = pd.concat([df for df in html_df if (df.shape[1] <= 11) and (df.shape[1] > 5)])

                if 'levels' in dir(df.columns):
                    df.columns = df.columns.get_level_values(1)
                else:
                    df = df[list(range(0, 10))]
                    column_index = df.index[(df[0] == '公司代號')][0]
                    df.columns = df.iloc[column_index]

                df['當月營收'] = pd.to_numeric(df['當月營收'], 'coerce')
                df = df[~df['當月營收'].isnull()]
                df = df[df['公司代號'] != '合計']

                df['date'] = datetime.date(self.date.year, self.date.month, 10)

                df = df.rename(columns={'公司代號': 'stock_id'})
                df = df.set_index(['stock_id', 'date'])

                data.append(df)
            except Exception as e:
                print(e)
                print('**WARRN: Pandas cannot find any table in the HTML file')
                return None
        df = pd.concat(data)
        if '備註' not in df.columns:
            df['備註'] = None
        df.iloc[:, 1:-1] = df.iloc[:, 1:-1].apply(lambda s: pd.to_numeric(s, errors='coerce'))
        df = df[df['公司名稱'] != '總計']
        df = df.where(pd.notnull(df), None)
        df = df.rename(columns={'公司名稱': "stock_name", "當月營收": "this_month_rev",
                                '上月營收': "last_month_rev", "去年當月營收": "last_year_rev",
                                '上月比較增減(%)': "cp_last_month_rev", "去年同月增減(%)": "cp_last_year_rev",
                                '當月累計營收': "cm_this_month_rev", "去年累計營收": "cm_last_month_rev",
                                '前期比較增減(%)': "cp_cm_rev", "備註": "note",
                                })
        df = df.reset_index()

        return df

In [27]:
'\xa0'

'\xa0'

In [ ]:
C=CrawlMonthlyRevnueTW(datetime.datetime(2020,4,1))
C.crawl_main()

In [ ]:
#pycharm 測試
import sys,os
sys.path.append("..")
import django
django.setup()
from crawlers.models import *
# from crawlers.finlab.pioneers import *
# from crawlers.finlab.import_tools import *

In [ ]:
add_to_sql(CompanyBasicInfoTW, df)